# **Global Multivariate Time Series Forecasting using XGBoost with Localized Lags**

In [20]:
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.ensemble import RandomForestClassifier
import joblib

In [21]:
df = pd.read_csv('/content/final_df.csv')

In [22]:
df.head()

,location_name,year,month,sand,silt,clay,soc,ph,bdod,cec,ndvi,t2m_c,td2m_c,rh_pct,tp_m,ssrd_jm2,lc_type1
0,Alexandria,2017,1,41.9,32.6,25.4,36.4,7.6,1.38,2.14,0.23490,13.499527,6.782051,63.804614,0.317607,5.313632e+09,50
1,Alexandria,2017,2,41.9,32.6,25.4,36.4,7.6,1.38,2.14,0.25945,14.102349,7.990648,66.637249,0.114083,6.185134e+09,50
2,Alexandria,2017,3,41.9,32.6,25.4,36.4,7.6,1.38,2.14,0.26755,16.424418,9.963948,65.578252,0.074304,2.725764e+09,50
3,Alexandria,2017,4,41.9,32.6,25.4,36.4,7.6,1.38,2.14,0.23695,18.480010,11.995997,65.915311,0.056938,1.058778e+10,50
4,Alexandria,2017,5,41.9,32.6,25.4,36.4,7.6,1.38,2.14,0.22175,22.038071,15.278302,65.490480,0.030640,1.216407e+10,50


In [23]:
# drop latitude, longitude
#df.drop(columns=["latitude", "longitude"], inplace=True)

In [24]:
df.columns

Index(['location_name', 'year', 'month', 'sand', 'silt', 'clay', 'soc', 'ph',
       'bdod', 'cec', 'ndvi', 't2m_c', 'td2m_c', 'rh_pct', 'tp_m', 'ssrd_jm2',
       'lc_type1'],
      dtype='object')

In [25]:
# rename LC_Type1 : lc_type1, area : location_name
#df = df.rename(columns={'LC_Type1': 'lc_type1', 'area': 'location_name'})

In [26]:
df['location_name'].value_counts()

,count
location_name,
Dakahlia,69519
Fayoum,30671
NorthSinai,16096
Matrouh,4385
NewValley,217
Manzala_Wetland,108
Burullus_Wetland,108
Siwa_Grasslands,108
Lake_Nasser_1,108


In [27]:
df = df.sort_values(["location_name", "year", "month"]).reset_index(drop=True)

In [28]:
df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

In [29]:
static_features = [
"sand", "silt", "clay", "soc", "ph",
"bdod", "cec", "lc_type1"
]

dynamic_features = [
"ndvi", "t2m_c", "td2m_c",
"rh_pct", "tp_m", "ssrd_jm2"
]

seasonal_features = ["month_sin", "month_cos"]

In [30]:
LAGS = [1, 2, 3]

for col in dynamic_features:
    for lag in LAGS:
        df[f"{col}_lag{lag}"] = (
            df.groupby("location_name")[col].shift(lag)
        )

In [31]:
df.isna().sum()

,0
location_name,0
year,0
month,0
sand,0
silt,0
clay,0
soc,0
ph,0
bdod,0
cec,0


In [32]:
df_ts = df.dropna().reset_index(drop=True)

In [33]:
# List of columns to use for forecasting
forecast_feature_cols = (
    dynamic_features +
    seasonal_features +
    [f"{col}_lag{lag}" for col in dynamic_features for lag in LAGS]
)

In [34]:
# Train a pipeline for each dynamic feature
forecast_models = {}

for target in dynamic_features:
    if target == "ssrd_jm2":
        y_f = np.log1p(df_ts[target])
    else:
        y_f = df_ts[target]

    train_mask = df_ts["year"] <= 2023
    test_mask  = df_ts["year"] > 2023

    X_train = df_ts.loc[train_mask, forecast_feature_cols]
    X_test  = df_ts.loc[test_mask, forecast_feature_cols]

    y_train = y_f[train_mask]
    y_test  = y_f[test_mask]

    model = XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    if target == "ssrd_jm2":
        pred = np.expm1(pred)
        y_true = np.expm1(y_test)
    else:
        y_true = y_test

    print(f"{target} MAE:", mean_absolute_error(y_true, pred))
    forecast_models[target] = model

ndvi MAE: 0.006590890680854667
t2m_c MAE: 0.08097648862405518
td2m_c MAE: 0.10097972217622081
rh_pct MAE: 0.424418293473741
tp_m MAE: 0.000978346017169657
ssrd_jm2 MAE: 42237199.711155154


In [35]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("ssrd_jm2 MAPE:", mape(y_true, pred), "%")

ssrd_jm2 MAPE: 0.4865374173512463 %


> Although MAE appears large due to the scale of ssrd_jm2, the MAPE of 0.49% indicates strong predictive performance.

> The regression model achieved a MAE of 4.22×10⁷ J/m² and a MAPE of 0.49%, indicating high prediction accuracy despite the large magnitude of the target variable.

In [36]:
# Save Forecasting
for feature, model in forecast_models.items():
    joblib.dump(model, f"xgb_forecast_{feature}.pkl")

| Feature    | Metric       | Reason |
|-----------|--------------|--------|
| ndvi      | MAE          | Small normalized values with limited scale variation |
| t2m_c     | MAE          | Temperature values are on a consistent and interpretable scale |
| td2m_c    | MAE          | Similar scale and distribution to temperature |
| rh_pct    | MAE          | Bounded percentage values (0–100) |
| tp_m      | MAE          | Small magnitude precipitation values |
| ssrd_jm2  | MAPE / NMAE  | Very large magnitude values; relative error is more meaningful than absolute error |


In [37]:
class FeatureFilter(BaseEstimator, TransformerMixin):
    def __init__(self, expected_features):
        self.expected_features = expected_features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X = X[[c for c in self.expected_features if c in X.columns]]
        for c in self.expected_features:
            if c not in X.columns:
                X[c] = np.nan
        return X[self.expected_features]

In [38]:
cls_model = joblib.load("xgb_desertification_pipeline.pkl")
le = joblib.load("label_encoder.pkl")
feature_order = joblib.load("feature_names.pkl")

In [39]:
# Forecast function + classification
def forecast_and_classify(df, location, years=5):
    months = years * 12
    last_row = df[df["location_name"]==location].sort_values(["year","month"]).iloc[-1].copy()
    forecasts = []
    year = last_row["year"]
    month = last_row["month"]

    for _ in range(months):
        month +=1
        if month>12:
            month=1
            year+=1

        last_row["month_sin"] = np.sin(2 * np.pi * month / 12)
        last_row["month_cos"] = np.cos(2 * np.pi * month / 12)

        row_pred = {"year": year, "month": month}

        for feature in dynamic_features:
            X_input = last_row[forecast_feature_cols].values.reshape(1, -1)
            pred = forecast_models[feature].predict(X_input)[0]
            if feature=="ssrd_jm2":
                pred = np.expm1(pred)
            row_pred[feature] = pred

            # update lags
            for lag in [3,2]:
                last_row[f"{feature}_lag{lag}"] = last_row[f"{feature}_lag{lag-1}"]
            last_row[f"{feature}_lag1"] = pred

        forecasts.append(row_pred)

    future_features = pd.DataFrame(forecasts)

    # add static features
    static_row = df[df["location_name"]==location].iloc[-1]
    for col in static_features:
        future_features[col] = static_row[col]

    # filter columns for classifier
    feature_filter = FeatureFilter(feature_order)
    X_future = feature_filter.transform(future_features)

    # predict classification
    pred_encoded = cls_model.predict(X_future)
    pred_labels = le.inverse_transform(pred_encoded)
    future_features["risk_level"] = pred_labels
    pred_proba = cls_model.predict_proba(X_future)
    future_features["risk_confidence"] = pred_proba.max(axis=1)

    final_forecast = future_features[["year","month"] + dynamic_features + ["risk_level","risk_confidence"]]
    return final_forecast

In [40]:
# Example forecast + classification
final_forecast = forecast_and_classify(df_ts, location="Dakahlia", years=5)
final_forecast.head()

,year,month,ndvi,t2m_c,td2m_c,rh_pct,tp_m,ssrd_jm2,risk_level,risk_confidence
0,2025,8,0.698545,29.301363,18.076424,50.246857,0.002520,1.228379e+10,Medium,0.973890
1,2025,9,0.690129,29.272839,18.059641,50.425446,0.002497,1.049342e+10,Medium,0.990247
2,2025,10,0.693093,28.836037,17.961786,50.626301,0.002496,1.159933e+10,Medium,0.978940
3,2025,11,0.682201,28.676203,17.879194,50.539280,0.002504,1.135944e+10,Medium,0.988638
4,2025,12,0.693725,28.541832,17.886631,50.834782,0.002477,1.148816e+10,Medium,0.986416


In [41]:
final_forecast.tail()

,year,month,ndvi,t2m_c,td2m_c,rh_pct,tp_m,ssrd_jm2,risk_level,risk_confidence
55,2030,3,0.686534,28.663282,17.857384,50.098244,0.002397,1.180195e+10,Medium,0.977304
56,2030,4,0.700634,29.257647,18.034309,50.086746,0.002409,1.230004e+10,Medium,0.973060
57,2030,5,0.698838,29.267040,18.071344,50.128792,0.002504,1.237629e+10,Medium,0.971392
58,2030,6,0.700259,29.273964,18.069414,50.190189,0.002507,1.238944e+10,Medium,0.971392
59,2030,7,0.697156,29.278467,18.078310,50.555744,0.002507,1.247119e+10,Medium,0.972638


In [42]:
final_forecast['risk_level'].value_counts()

,count
risk_level,
Medium,60


## Model Limitations

1. **Requires recent historical data for the selected location**  
   - The forecasting model relies on the last 3 months of historical data for each location to generate accurate predictions.  
   - Without recent data, the predictions may be unreliable.

2. **Best results on trained locations**  
   - The model was trained on samples from various Egyptian governorates, covering most common soil and environmental characteristics.  
   - It **can provide approximate predictions for new locations** with similar soil and climate conditions, but the accuracy may be lower than for locations included in the training data.